# Common以下のコード補足

In [1]:
import numpy as np

***
# functions

## softmax関数
xが1次元，2次元の場合に分けて，オーバーフロー対策のために各データから最大値を引く

In [2]:
x = np.random.rand(3, 10)
print(x)
print(x.max(axis=1, keepdims=True))
print(x - x.max(axis=1, keepdims=True))

[[0.2940373  0.7004095  0.36041282 0.20236627 0.54087628 0.66112017
  0.59811509 0.76990321 0.05268572 0.17192662]
 [0.17896811 0.69265766 0.40411021 0.92167682 0.85305711 0.29441756
  0.00198469 0.11971135 0.32184432 0.64175278]
 [0.4372709  0.28385779 0.95319899 0.55491942 0.12182214 0.72131566
  0.56105273 0.39637394 0.52347445 0.82033648]]
[[0.76990321]
 [0.92167682]
 [0.95319899]]
[[-0.47586591 -0.0694937  -0.40949039 -0.56753694 -0.22902693 -0.10878303
  -0.17178812  0.         -0.71721748 -0.59797659]
 [-0.74270871 -0.22901916 -0.51756661  0.         -0.06861971 -0.62725926
  -0.91969213 -0.80196547 -0.5998325  -0.27992404]
 [-0.51592809 -0.66934119  0.         -0.39827957 -0.83137685 -0.23188333
  -0.39214625 -0.55682505 -0.42972454 -0.13286251]]


## cross_entropy_error
- 1データ分
$$
L = - \Sigma_k t_k \log y_k
$$

- バッチ
$$
L = - \frac{1}{N} \Sigma_n \Sigma_k t_{nk} \log y_{nk}
$$

- targetラベルがone-hot-vectorであれば，正解ラベルに対応するlog yを計算するだけ．

In [10]:
# 教師データのone-hot-vectorを正解ラベルのインデックスに変換
t = np.array([[0, 0, 1, 0],
              [1, 0, 0, 0],
              [0, 0, 0, 1]])  # target
y = np.random.rand(*t.shape)  # 入力
print(f'y_org: \n {y}')
print(f't_org: \n {t}')

batch_size = y.shape[0]  # バッチ

t = t.argmax(axis=1)  # targetをone-hot-vectorからインデックスに変換

out = -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size
print(f'out: \n {out}')
print(f't_argmax: \n {t}')

y_org: 
 [[0.45392039 0.41359739 0.44500178 0.45643463]
 [0.09015554 0.79384601 0.86955705 0.23079442]
 [0.15610483 0.08647592 0.11261886 0.82418443]]
t_org: 
 [[0 0 1 0]
 [1 0 0 0]
 [0 0 0 1]]
out: 
 1.1364184696268513
t_argmax: 
 [2 0 3]


***
# layers

## SigmoidWithLoss
$$
y = \frac{1}{1 + \exp(-x)}
\\ \\
L = - \{ t \log y + (1 - t) \log(1 - y) \}
$$
- CBOWモデルの高速化(多値分類を二値分類に置き換える)で使用．2クラス分類．
- 答えが不正解のときt = 0, 正解のときt = 1が渡される．

In [11]:
x = np.random.rand(5).reshape(5, 1)  # 入力
y = 1 / (1 + np.exp(-x))  # Sigmoid
# 以下の配列と t = 0 or 1 のラベルをcross_entropy_errorに渡せばよい
print(y)
print(np.c_[1 - y, y])

[[0.6514761 ]
 [0.58336691]
 [0.72787286]
 [0.58471967]
 [0.5827764 ]]
[[0.3485239  0.6514761 ]
 [0.41663309 0.58336691]
 [0.27212714 0.72787286]
 [0.41528033 0.58471967]
 [0.4172236  0.5827764 ]]


## Embedding
- CBOWモデルの高速化．入力層->中間層の全結合層MatMulレイヤの代わりに使う．
- MuｔMulでやったことは単語IDのone-hot-vectorと重みW_inの積和で，これはW_inから単語IDに対応するインデックスの行を抜き出すことに等しい

In [46]:
# forwardメソッド
W = np.random.rand(5, 3)  # 重み
contexts = np.array([[0, 2],
                     [1, 3],
                     [2, 4],
                     [3, 0],
                     [4, 2]])  # window_size=2, n=6のコンテキスト

print(f'W: \n {W}')

# コンテキストの数と同じだけEmbeddingレイヤを使ってコンテキストの列ごとに処理する
for i in range(contexts.shape[1]) :
    idx = contexts[:, i]
    out = W[idx]
    print(f'out_{i}: \n {out}')

W: 
 [[0.82088295 0.8341825  0.79070042]
 [0.59844225 0.7949682  0.31591123]
 [0.47798136 0.18138265 0.93391213]
 [0.81411959 0.947904   0.97367685]
 [0.78346381 0.45951483 0.23582957]]
out_0: 
 [[0.82088295 0.8341825  0.79070042]
 [0.59844225 0.7949682  0.31591123]
 [0.47798136 0.18138265 0.93391213]
 [0.81411959 0.947904   0.97367685]
 [0.78346381 0.45951483 0.23582957]]
out_1: 
 [[0.47798136 0.18138265 0.93391213]
 [0.81411959 0.947904   0.97367685]
 [0.78346381 0.45951483 0.23582957]
 [0.82088295 0.8341825  0.79070042]
 [0.47798136 0.18138265 0.93391213]]


In [51]:
# backwardメソッド
grad = np.ones_like(W)  # 勾配
dW = grad
dW[...] = 0  # 勾配を初期化

dout = np.arange(W.size).reshape(W.shape)
print(f'dW_init: \n {dW}')
print(f'dout: \n {dout}')
print(f'idx: \n {contexts[:, 1]}')

np.add.at(dW, contexts[:, 1], dout)  # dWの指定した行にdoutの各行を足していく
print(f'dW: \n {dW}')

dW_init: 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
dout: 
 [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]]
idx: 
 [2 3 4 0 2]
dW: 
 [[ 9. 10. 11.]
 [ 0.  0.  0.]
 [12. 14. 16.]
 [ 3.  4.  5.]
 [ 6.  7.  8.]]


***
# trainer
## Trainer

In [35]:
# データのシャッフル
np.random.permutation(10)

array([6, 1, 3, 4, 0, 9, 7, 5, 2, 8])

In [39]:
# fitメソッド
x = np.random.randn(1000)
data_size = len(x)
batch_size = 32
max_iters = data_size // batch_size  # 31
max_epoch = 10
eval_interval = 20

for epoch in range(max_epoch):
    print(f'--------------epoch: {epoch}--------------')
    print('ここでデータをシャッフル')
    
    for iters in range(max_iters):
        print(f'--------iters: {iters}--------')
        print('ミニバッチを取得')
        print('順伝播で損失計算')
        print('逆伝播で勾配計算')
        print('max_gradを指定していれば勾配クリッピング')
        print('パラメータ更新 (optimizer.update)')
        print('total_lossに損失の値を足し上げる')
        print('loss_countに損失関数の計算回数をカウント')
        
        if (eval_interval is not None) and (iters % eval_interval) == 0:
            print('------eval------')
            print('損失の平均を計算，出力，リスト追加')
            print('total_loss, loss_countを初期化')

--------------epoch: 0--------------
ここでデータをシャッフル
--------iters: 0--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
------eval------
損失の平均を計算，出力，リスト追加
total_loss, loss_countを初期化
--------iters: 1--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 2--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 3--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 4--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 5--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
lo

total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 24--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 25--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 26--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 27--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 28--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 29--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 30--------
ミニバッチを取得

パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
------eval------
損失の平均を計算，出力，リスト追加
total_loss, loss_countを初期化
--------iters: 21--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 22--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 23--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 24--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 25--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 26--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.u